## Import Library
Import all libraries required to run the functional genetic algorithm

In [1]:
import os
import shutil
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import pip
pip.main(["install", "openpyxl"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: openpyxl in ./virtualEnv/lib/python3.12/site-packages (3.1.5)

Requirement already satisfied: et-xmlfile in ./virtualEnv/lib/python3.12/site-packages (from openpyxl) (1.1.0)

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

0

## City & Route Class

The City class to make it easier to define cities coordinate and calculate the distance between two cities.

The Route class to determine the route taken and calculate the total distance and fitness value

In [2]:
class City:
    def __init__(self,name, x, y):
        self.x = x
        self.y = y
        self.name = name

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((abs(self.x - city.x) ** 2) + (abs(self.y - city.y)** 2))
        return distance

    def __repr__(self):
        return "("+str(self.name)+")"

In [3]:
%%capture cap
class Route:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness = 0.0

    def routeDistance(self):
        if self.distance == 0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None

                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]

                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

    def printCity(self):
        namaKota =[]
        for i in range(0,len(self.route)):
            namaKota.append(self.route[i].name)
        return "("+str(namaKota)+ ")"

## Genetic Algorithm Functions

Create several functions to perform each step in the genetic algorithm, such as:
- Create Route
- Initial Population
- Rank Routes
- Selection
- Check Tabulist
- Crossover
- Mutate
- Update Generation

### Create Route

In [4]:
def createRoute(cityList):
    #Randomize city based on city in dataset to assign into route
    route = random.sample(cityList, len(cityList))
    return route

### Intialization the Population

In [5]:
def initialPopulation(popSize, cityList):
    population = []
    # Create population based on population size
    for i in range(0, popSize):
      route = createRoute(cityList)
      while route in population:
        route = createRoute(cityList)
      population.append(createRoute(cityList))
    return population

### Rank Routes

In [6]:
def rankRoutes(population):
    fitnessResults = []
    rankRoute = {}

    #Determine fitness each individual in population
    for i in range(0, len(population)):
        fitnessResults.append(Route(population[i]).routeFitness())

    #Sorting based on fitness value
    sortedFitness = sorted(fitnessResults, reverse=True)
    for i in range(0, len(sortedFitness)):
        for j in range(0, len(population)):
            if sortedFitness[i] == Route(population[j]).routeFitness():
                rankRoute[i] = population[j]

    return rankRoute

### Selection

In [7]:
def selection(population, DEBUG=False):
    selection = []
    #Take first 2  the best individuals
    for i in range(0,2):
        selection.append(population[i])
        if DEBUG:
            print("seleksi ke-" + str(i+1) + " " + str(Route(selection[i]).printCity()) + "\n" +
                " Jarak = " + str(Route(selection[i]).routeDistance()) + "\n" + " Fitness = " + str(
                Route(selection[i]).routeFitness()))
    return selection

### Tabulist

In [8]:
def tabulist(selectionResults,tabulist):
    isTabulist1 = False
    isTabulist2 = False
    isTabulist = False

    #Check tabulist
    for j in range(0,len(tabulist)):
        if selectionResults[0] == tabulist[j]:
            isTabulist1 = True
        if selectionResults[1] == tabulist[j]:
            isTabulist2 = True

    if isTabulist1 == False:
        tabulist.append(selectionResults[0])
    if isTabulist2 == False:
        tabulist.append(selectionResults[1])

    if isTabulist1 == True and isTabulist2 == True:
        isTabulist = True

    return isTabulist,tabulist

### Crossover

In [9]:
def crossover(parent1, parent2,tab, DEBUG=False):
    childP1 = []
    childP2 = []
    child = []

    # randomizes the gene number to be carried out crossover
    # if tabulis is true then the crossover result is the same as parent1 (best individual)
    if tab == True:
        geneA = 0
        geneB = len(parent1)-1
    else:
        geneA = random.randint(1, len(parent1)-1)
        geneB = random.randint(1, len(parent1)-1)
        # Gene A and Gene B do not have the same value
        while geneA == geneB:
            geneB = random.randint(1, len(parent1)-1)

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    if DEBUG:
        print("stargene = "+str(startGene+1)+", endGene = "+str(endGene+1))

    for i in range(startGene, endGene+1):
        childP1.append(parent1[i])

    for item in parent2:
        if item not in childP1:
            childP2.append(item)

    #Crossover Result
    idxChild1 = 0
    idxChild2 = 0
    for i in range(len(parent1)):
      if i >= startGene and i <= endGene:
        child.append(childP1[idxChild1])
        idxChild1 += 1
      else:
        child.append(childP2[idxChild2])
        idxChild2 += 1

    return child

### Mutate

In [10]:
def mutate(individual,DEBUG=False):
    #randomizes the gene number to be exchanged
    swapped = random.randint(1, len(individual)-1)
    swapWith = random.randint(1, len(individual)-1)
    #swapped and swapWith cannot have the same value
    while swapped == swapWith:
        swapWith = random.randint(1, len(individual)-1)

    city1 = individual[swapped]
    city2 = individual[swapWith]

    if DEBUG:
        print("swapped = "+str(swapped+1) + " , swap with = " + str(swapWith+1))

    #hasil mutasi
    individual[swapped] = city2
    individual[swapWith] = city1
    return individual

### Update Generation

In [11]:
def updateGeneration(population,popsize,mutate,DEBUG=False):
    generasibaru =[]
    #combining populations with mutation results to create a new generation
    for i in range(0,popsize):
        generasibaru.append(population[i])

    #replace the individual with the smallest fitness (last index) with the mutation result if the mutation result is greater
    if Route(mutate).routeFitness() >= Route(generasibaru[-1]).routeFitness():
      generasibaru[popsize-1] =mutate

    if DEBUG:
        for i in range(0,popsize):
            print("Generasi baru ke-" + str(i+1) + " " + str(Route(generasibaru[i]).printCity()) +
                " Jarak = " + str(Route(generasibaru[i]).routeDistance()) + " Fitness = " + str(
                Route(generasibaru[i]).routeFitness()))

    return generasibaru

### Implement The Genetica Algorithm

In [12]:
def geneticAlgorithm(population, popSize, generations, imageFilename='algen_result.png' ,DEBUG=False):
    tabulistResult = []
    finalPopulation = []
    pop = []

    # Create Population
    populasi = initialPopulation(popSize, population)

    # Sort each individual in population based on ranking
    rankPopulasi = rankRoutes(populasi)
    for i in range(0, popSize):
        pop.append(rankPopulasi[i])

    # Start genetic algorithm with the first generation
    print("=========================== GENERASI KE " + str(1) + "===============================================================>")
    for i in range(0,len(pop)):
        print("populasi ke-" + str(i+1) + " " + str(Route(pop[i]).printCity()) +
              " Jarak = " + str(Route(pop[i]).routeDistance()) + " Fitness = " + str(
            Route(pop[i]).routeFitness()))
    print("==========================================================================================>")

    bestDistance = 9999999999999
    progress =[]
    for i in range(0, generations):
        nextGeneration = []

        #Selection process
        selectionResults = selection(pop, DEBUG)

        #Check Tabulist
        tab, tabulistResult = tabulist(selectionResults, tabulistResult)

        #Crossover if the tabulist true
        crossoverResult = crossover(selectionResults[0], selectionResults[1], tab, DEBUG)
        if DEBUG:
            if tab == False:
                print("Tabulist False")
                print("==========================================================================================>")
                print("hasil crossover = " + str(crossoverResult))
                print("==========================================================================================>")
            elif tab == True:
                print("Tabulist true")
                print("==========================================================================================>")
                print("seleksi 1 = " + str(crossoverResult))
                print("==========================================================================================>")
            print("TABULIST : ", tabulistResult, "\n")

        #Mutate Process
        children = mutate(crossoverResult, DEBUG)
        if DEBUG:
            print("hasil mutasi = " + str(children) + " Jarak = " + str(Route(children).routeDistance()) + " Fitness = " + str(
                Route(children).routeFitness()))
            print("==========================================================================================>")

        #Update Generation
        nextGeneration = updateGeneration(pop, popSize, children, DEBUG)
        poptemp = nextGeneration

        # Sorting new generations based on fitness
        poptemp = rankRoutes(poptemp)
        pop = []
        for j in range(0, popSize):
            pop.append(poptemp[j])

        # Save the best individual in this generation
        tempbestroute = Route(pop[0]).routeDistance()
        progress.append(Route(pop[0]).routeDistance())
        if (bestDistance > tempbestroute):
            bestIndividu = Route(pop[0]).printCity()
            bestDistance = Route(pop[0]).routeDistance()
            bestFitness = Route(pop[0]).routeFitness()

        # Next Generation process
        finalPopulation = pop
        if (DEBUG and i < generations-1) or (i == generations-1):
            print("\n====================================== GENERASI KE " + str(
                i + 2) + "======================================>")
            for n in range(0, len(pop)):
                print("populasi ke-" + str(n+1) + " " + str(Route(pop[n]).printCity()) +
                    " Jarak = " + str(Route(pop[n]).routeDistance()) + " Fitness = " + str(
                    Route(pop[n]).routeFitness()))
            print("=========================================================================================>")


    print("BEST INDIVIDU = " + str(bestIndividu))
    print("BEST DISTANCE = " + str(bestDistance))
    print("BEST FITNESS = " + str(bestFitness))

    # Visualize the genetic algorithm result
    plt.figure(0)
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.title('Genetica Algorithm Result')
    plt.savefig(imageFilename)
    plt.cla()
    plt.clf()

    return finalPopulation, bestDistance

## Ant Colony Optimization Function

In [13]:
def antColonyOptimization(city, iteration, nAnts, rho, alpha, beta, initialPheromne, routes, imageFilename='aco_result.png', DEBUG=False):
    cityList = []
    for i in range(0, len(city)):
        cityList.append(City(name = city.iloc[i,0],x=city.iloc[i][1],y=city.iloc[i][2]))

    distances = np.zeros((len(cityList), len(cityList)))
    visibility = np.zeros((len(cityList), len(cityList)))

    for row in range(len(cityList)):
        for col in range (len(cityList)):
            distance = cityList[row].distance(cityList[col])
            distances[row, col] = distance
            visibility[row, col] = 1/distance  if distance != 0 else 0

    # Calculate total distance in Genetica algorithm result
    totalDistance = np.zeros((len(routes), 1)) # initiate total distance
    print(f"Genetica Algorithm Result: ")
    for i in range(len(routes)):
        routeStr = ""
        distance = 0
        for j in range(len(routes[i])-1):
            distance += routes[i][j].distance(routes[i][j+1])
            routeStr += " "+str(routes[i][j].name)
        routeStr += " "+str(routes[i][-1].name)
        print(f"Ant {i+1}: [{routeStr}] Distance: {Route(routes[i]).routeDistance()}")
        totalDistance[i] = distance

    # Inisialization Pheromne
    pheromne = initialPheromne * np.ones((len(cityList), len(cityList)))
    # Update pheromne with genetica algorithm result
    pheromne = (1 - rho) * pheromne # Evaporation
    for i in range(len(routes)):
        delta = 1 / totalDistance[i][0] # Delta Pheromne
        for j in range(len(cityList)):
            pheromne[int(routes[i][j].name) - 1, int(routes[i][j+1].name) - 1] += delta
            pheromne[int(routes[i][j + 1].name) - 1, int(routes[i][j].name) - 1] += delta

    print("\nInitial Pheromne")
    print("--------------------------------")
    print("Initail City | Destination City | Pheromne")
    for row in range(len(cityList)):
        for col in range (row+1,len(cityList)):
            print(f"{row + 1}             | {col + 1}               | {pheromne[row, col]:.4f}")
    print("------------------------------------------\n")

    bestRoute = None
    bestDistance = float('inf')
    bestDistances = []

    # ACO Iteration
    for idx in range(iteration):
        if (DEBUG and idx < iteration-1) or (idx == iteration-1):
            print(f"=========================== Iteration {idx+1} ============================")
        antAndDistanceStr = ""

        routes = np.ones((nAnts, len(cityList)+1), dtype=int)
    
        # Randomize the first city each ants
        initialCitiesIdx = np.random.permutation(len(cityList))
        totalDistance = np.zeros((nAnts, 1))
        for i in range(nAnts):
            distance = 0
            routes[i, 0] = initialCitiesIdx[i] + 1 # Assign first city to routes
            visibilityTemp = np.array(visibility)

            for j in range(len(cityList)-1):

                # Calculate Probabilities
                currentLocation = int(routes[i, j] -1)
                visibilityTemp[:, currentLocation] = 0

                pFeature = np.power(pheromne[currentLocation, :], beta)
                vFeature = np.power(visibilityTemp[currentLocation, :], alpha)
                features = np.multiply(pFeature, vFeature)
                total = np.sum(features)
                probabilities = features/total
                
                if DEBUG:
                    print(f"Ant {i + 1}: {routes[i, :]}")
                    print("---------------------")
                    print("City  | Probability |")
                    for k in range(len(cityList)):
                        print(f"{k + 1}     | {probabilities[k]:.4f}       |")
                    print("---------------------")

                # Choose next city with highest probability
                nextCityIdx = np.argmax(probabilities)
                routes[i, j+1] = nextCityIdx + 1 # Add next city to route

                distance += distances[int(routes[i, j]) - 1, int(routes[i, j+1]) - 1]


            routes[i, -1] = routes[i, 0] # Back to first City
            if DEBUG:
                print(f"Ant {i + 1}: {routes[i, :]}")

            # Calculate last city to first city
            distance += distances[int(routes[i, -2]) - 1, int(routes[i, -1]) - 1]
            totalDistance[i] = distance
            antAndDistanceStr += f"Ant {i+1}: {'-'.join(map(str, map(int, routes[i, :])))} | Distance = {totalDistance[i, 0]:.4f}\n"
            if DEBUG:
                print("\n====================\n")

        if DEBUG:
            print(f"Iteration {idx+1} Result: ")
            print(antAndDistanceStr)


        # Search the best routes
        distanceMinIdx = np.argmin(totalDistance)
        distanceMin = totalDistance[distanceMinIdx]
        if distanceMin < bestDistance:
            bestDistance = distanceMin
            bestRoute = routes[distanceMinIdx, :]

        bestDistances.append(bestDistance)

        # Update pheromne
        pheromne = (1 - rho) * pheromne # Evaporation
        for i in range(nAnts):
            delta = 1 / totalDistance[i][0] # Delta Pheromne
            for j in range(len(cityList)):
                pheromne[int(routes[i, j]) - 1, int(routes[i, j+1]) - 1] += delta
                pheromne[int(routes[i, j + 1]) - 1, int(routes[i, j]) - 1] += delta

        if (DEBUG and idx < iteration-1) or (idx == iteration-1):
            print("Update Pheromne")
            print("--------------------------------")
            print("Initail City | Destination City | New Pheromne")
        for i in range(len(cityList)):
            for j in range(i+1, len(cityList)):
                pheromneValue = pheromne[i, j]
                if (DEBUG and idx < iteration-1) or (idx == iteration-1):
                    print(f"{i + 1}             | {j + 1}               | {pheromneValue:.4f}")
        if (DEBUG and idx < iteration-1) or (idx == iteration-1):
            print("------------------------------------------\n")


    print(f"The best routes: {'-'.join(map(str, map(int, bestRoute)))} | Total Distance = {bestDistance[0]:.4f}")

    # Ploting ACO Result
    plt.figure(1)
    plt.plot(range(1, iteration + 1), bestDistances)
    plt.xlabel('Iteration')
    plt.ylabel('Distance')
    plt.title('Ant Colony Optimization Result')
    plt.savefig(imageFilename)
    plt.cla()
    plt.clf()

    return bestDistances[-1]

# Test Case Main

In [16]:
%%capture cap --no-stderr

if os.path.exists('imageResult'):
    shutil.rmtree('imageResult')
    os.mkdir('imageResult')
else:
    os.mkdir('imageResult')

if os.path.exists('testcase_result.xlsx'):
    os.remove('testcase_result.xlsx')

DEBUG = False

testcase = pd.read_excel('testcase.xlsx')
testcase['Image GA Percobaan 1'] = testcase['Image GA Percobaan 1'].astype(str)
testcase['Image GA Percobaan 2'] = testcase['Image GA Percobaan 2'].astype(str)
testcase['Image GA Percobaan 3'] = testcase['Image GA Percobaan 3'].astype(str)
testcase['Image GA-ACO Percobaan 1'] = testcase['Image GA-ACO Percobaan 1'].astype(str)
testcase['Image GA-ACO Percobaan 2'] = testcase['Image GA-ACO Percobaan 2'].astype(str)
testcase['Image GA-ACO Percobaan 3'] = testcase['Image GA-ACO Percobaan 3'].astype(str)

for idx in range(0,len(testcase)):
    dataset = testcase.loc[idx,"Dataset"]

    # Algen Parameters
    populationSize = testcase.loc[idx,"Populasi"]
    generation = testcase.loc[idx,"Generasi"]

    # ACO Parameters
    iteration = testcase.loc[idx,"Generasi"]
    nAnts = testcase.loc[idx,"Jumlah Semut"]
    rho = testcase.loc[idx,"Rho"]
    alpha = testcase.loc[idx,"Alpha"]
    beta = testcase.loc[idx,"Beta"]
    initialPheromne = testcase.loc[idx,"Pheromone Awal"]

    

    city = pd.read_csv("./dataset/"+dataset, header=None, sep=' ')
    cityList = []
    for i in range(0,len(city)):
        cityList.append(City(name = city.iloc[i,0],x=city.iloc[i][1],y=city.iloc[i][2]))

    ## Test 1
    GAImageFilename1 = './imageResult/'+dataset.replace('.csv','')+'_1_GA_'+str(generation)+'.png'
    ACOImageFilename1 = './imageResult/'+dataset.replace('.csv','')+'_1_ACO_'+str(generation)+'.png'

    start_time1 = time.time()
    # Start Genetic Algorithm process
    print("\n====================================== Genetica Algorithm ======================================\n")
    print(f"Population Size: {str(populationSize)}")
    print(f"Generation: {str(generation)}")
    print()
    result1, GADistance1 = geneticAlgorithm(population=cityList, popSize=populationSize, generations=generation, imageFilename=GAImageFilename1, DEBUG=DEBUG)

    end_algen_time1 = time.time()
    algo_time1 = end_algen_time1 - start_time1

    # add first city to last index in each route
    newPop1 = []
    for re1 in result1:
        firstCity1 = re1[0]
        newRoute1 = []
        for r in re1:
            newRoute1.append(r)
        newRoute1.append(firstCity1)
        newPop1.append(newRoute1)

    # Start Ant Colony Optimization Algorithm process
    ACODistance1 = antColonyOptimization(city=city, iteration=iteration, nAnts=nAnts, rho=rho, alpha=alpha, beta=beta, initialPheromne=initialPheromne, routes=newPop1, imageFilename=ACOImageFilename1 ,DEBUG=DEBUG)
    aco_time1 = time.time() - end_algen_time1
    executionTime1 = time.time() - start_time1

    testcase.loc[idx, "Jarak GA Percobaan 1"] = GADistance1
    testcase.loc[idx, "Jarak GA-ACO Percobaan 1"] = ACODistance1

    testcase.loc[idx, "Runtime GA Percobaan 1"] = round(algo_time1/60,4)
    testcase.loc[idx, "Runtime GA-ACO Percobaan 1"] = round(aco_time1/60,4)

    testcase.loc[idx, "Image GA Percobaan 1"] = GAImageFilename1
    testcase.loc[idx, "Image GA-ACO Percobaan 1"] = ACOImageFilename1


    ## Test 2
    GAImageFilename2 = './imageResult/'+dataset.replace('.csv','')+'_2_GA_'+str(generation)+'.png'
    ACOImageFilename2 = './imageResult/'+dataset.replace('.csv','')+'_2_ACO_'+str(generation)+'.png'

    start_time2 = time.time()
    # Start Genetic Algorithm process
    print("\n====================================== Genetica Algorithm ======================================\n")
    print(f"Population Size: {str(populationSize)}")
    print(f"Generation: {str(generation)}")
    print()
    result2, GADistance2 = geneticAlgorithm(population=cityList, popSize=populationSize, generations=generation, imageFilename=GAImageFilename2, DEBUG=DEBUG)

    end_algen_time2 = time.time()
    algo_time2 = end_algen_time2 - start_time2

    # add first city to last index in each route
    newPop2 = []
    for re2 in result2:
        firstCity2 = re2[0]
        newRoute2 = []
        for r in re2:
            newRoute2.append(r)
        newRoute2.append(firstCity2)
        newPop2.append(newRoute2)

    # Start Ant Colony Optimization Algorithm process
    ACODistance2 = antColonyOptimization(city=city, iteration=iteration, nAnts=nAnts, rho=rho, alpha=alpha, beta=beta, initialPheromne=initialPheromne, routes=newPop2, imageFilename=ACOImageFilename2 ,DEBUG=DEBUG)
    aco_time2 = time.time() - end_algen_time2
    executionTime2 = time.time() - start_time2

    testcase.loc[idx, "Jarak GA Percobaan 2"] = GADistance2
    testcase.loc[idx, "Jarak GA-ACO Percobaan 2"] = ACODistance2

    testcase.loc[idx, "Runtime GA Percobaan 2"] = round(algo_time2/60,4)
    testcase.loc[idx, "Runtime GA-ACO Percobaan 2"] = round(aco_time2/60,4)

    testcase.loc[idx, "Image GA Percobaan 2"] = GAImageFilename2
    testcase.loc[idx, "Image GA-ACO Percobaan 2"] = ACOImageFilename2


    ## Test 3
    GAImageFilename3 = './imageResult/'+dataset.replace('.csv','')+'_3_GA_'+str(generation)+'.png'
    ACOImageFilename3 = './imageResult/'+dataset.replace('.csv','')+'_3_ACO_'+str(generation)+'.png'

    start_time3 = time.time()
    # Start Genetic Algorithm process
    print("\n====================================== Genetica Algorithm ======================================\n")
    print(f"Population Size: {str(populationSize)}")
    print(f"Generation: {str(generation)}")
    print()
    result3, GADistance3 = geneticAlgorithm(population=cityList, popSize=populationSize, generations=generation, imageFilename=GAImageFilename3, DEBUG=DEBUG)

    end_algen_time3 = time.time()
    algo_time3 = end_algen_time3 - start_time3

    # add first city to last index in each route
    newPop3 = []
    for re3 in result3:
        firstCity3 = re3[0]
        newRoute3 = []
        for r in re3:
            newRoute3.append(r)
        newRoute3.append(firstCity3)
        newPop3.append(newRoute3)

    # Start Ant Colony Optimization Algorithm process
    ACODistance3 = antColonyOptimization(city=city, iteration=iteration, nAnts=nAnts, rho=rho, alpha=alpha, beta=beta, initialPheromne=initialPheromne, routes=newPop3, imageFilename=ACOImageFilename3 ,DEBUG=DEBUG)
    aco_time3 = time.time() - end_algen_time3
    executionTime3 = time.time() - start_time3

    testcase.loc[idx, "Jarak GA Percobaan 3"] = GADistance3
    testcase.loc[idx, "Jarak GA-ACO Percobaan 3"] = ACODistance3

    testcase.loc[idx, "Runtime GA Percobaan 3"] = round(algo_time3/60,4)
    testcase.loc[idx, "Runtime GA-ACO Percobaan 3"] = round(aco_time3/60,4)

    testcase.loc[idx, "Image GA Percobaan 3"] = GAImageFilename3
    testcase.loc[idx, "Image GA-ACO Percobaan 3"] = ACOImageFilename3

testcase.to_excel('testcase_result.xlsx', index=False)